In [ ]:
import numpy as np


ACTIONS = ["ImmediateEvacuation", "DelayAndObserve", "SendDroneFirst", "PreemptiveRelocation"]
A_EVAC = 0
A_DELAY = 1
A_DRONE = 2
A_RELOC = 3


MAX_HAZARD = 9
MAX_RESOURCES = 9
STATUS_LEVELS = 4


H_HIGH_THRESHOLD = 7
H_MED_THRESHOLD = 3


STATES = []
for h in range(MAX_HAZARD + 1):
    for r in range(MAX_RESOURCES + 1):
        for s in range(STATUS_LEVELS):
            STATES.append((h, r, s))

def get_state_id(h, r, s):

    h = max(0, min(MAX_HAZARD, h))
    r = max(0, min(MAX_RESOURCES, r))
    s = max(0, min(STATUS_LEVELS - 1, s))


    return (h * (MAX_RESOURCES + 1) * STATUS_LEVELS) + (r * STATUS_LEVELS) + s

class DisasterMDPModel:
    def __init__(self):
        self.num_states = len(STATES)
        self.num_actions = len(ACTIONS)
        self.gamma = 0.9
        self.V = np.zeros(self.num_states)

    def get_reward(self, state_tuple, action, next_state_tuple):
        h, r, s = state_tuple
        nh, nr, ns = next_state_tuple

        reward = 0


        if ns == 3 and s != 3:
            if h >= H_HIGH_THRESHOLD:
                reward += 100
            elif h >= H_MED_THRESHOLD:
                reward += 80
            else:
                reward += 10


            if action == A_RELOC and h >= H_MED_THRESHOLD and h < H_HIGH_THRESHOLD:
                reward += 20

        if nr < r:
            reward -= (r - nr) * 5


        if action == A_DELAY:
            if h >= H_HIGH_THRESHOLD: reward -= 50
            if h < H_MED_THRESHOLD: reward += 5
            if nh > h: reward -= 20

        if action == A_DRONE:
            reward -= 2
            if h >= H_MED_THRESHOLD and h < H_HIGH_THRESHOLD: reward += 5

        if nh >= H_HIGH_THRESHOLD and ns == 2:
            reward -= 50
        if nr == 0 and nh > 0:
            reward -= 100

        return reward

    def get_transition_probs(self, state_id, action):
        h, r, s = STATES[state_id]
        transitions = []


        if s == 3: return [(1.0, state_id, 0)]
        if r == 0 and h >= H_HIGH_THRESHOLD: return [(1.0, state_id, 0)]


        if action == A_EVAC:

            nr = max(0, r - 3)
            ns = 3
            nid = get_state_id(h, nr, ns)
            rew = self.get_reward((h,r,s), action, STATES[nid])
            transitions.append((1.0, nid, rew))


        elif action == A_DELAY:
            if h < H_MED_THRESHOLD:

                nid_esc = get_state_id(min(MAX_HAZARD, h + 1), r, 1 if h+1 >= H_MED_THRESHOLD else s)
                rew_esc = self.get_reward((h,r,s), action, STATES[nid_esc])
                transitions.append((0.1, nid_esc, rew_esc))


                transitions.append((0.7, state_id, self.get_reward((h,r,s), action, STATES[state_id])))


                nid_safe = get_state_id(0, r, s)
                transitions.append((0.2, nid_safe, 20))

            else:

                nh_w = min(MAX_HAZARD, h + 1)
                ns_w = 2 if (nh_w >= H_HIGH_THRESHOLD and s == 1) else s
                nid_w = get_state_id(nh_w, r, ns_w)
                rew_w = self.get_reward((h,r,s), action, STATES[nid_w])
                transitions.append((0.4, nid_w, rew_w))


                nh_i = max(0, h - 1)
                nid_i = get_state_id(nh_i, r, s)
                rew_i = self.get_reward((h,r,s), action, STATES[nid_i])
                transitions.append((0.2, nid_i, rew_i))


                transitions.append((0.4, state_id, self.get_reward((h,r,s), action, STATES[state_id])))


        elif action == A_DRONE:

            nr = max(0, r - 1)


            nid_better = get_state_id(max(0, h - 1), nr, s)
            rew_better = self.get_reward((h,r,s), action, STATES[nid_better])
            transitions.append((0.3, nid_better, rew_better))


            nid_same = get_state_id(h, nr, s)
            rew_same = self.get_reward((h,r,s), action, STATES[nid_same])
            transitions.append((0.7, nid_same, rew_same))


        elif action == A_RELOC:

            if s == 1 or (h >= H_MED_THRESHOLD and h < H_HIGH_THRESHOLD):
                nr = max(0, r - 2)
                ns = 3
                nid = get_state_id(h, nr, ns)
                rew = self.get_reward((h,r,s), action, STATES[nid])
                transitions.append((1.0, nid, rew))
            else:

                nr = max(0, r - 2)
                nid = get_state_id(h, nr, s)
                rew = self.get_reward((h,r,s), action, STATES[nid])
                transitions.append((1.0, nid, rew))

        return transitions

    def value_iteration(self, epsilon=0.001):
        print(f"{'Iteration':<10} | {'Max Change':<15}")
        print("-" * 30)

        iteration = 0
        while True:
            delta = 0
            new_V = np.copy(self.V)


            for s in range(self.num_states):
                if STATES[s][2] == 3: continue

                q_values = []
                for a in range(self.num_actions):
                    q_val = 0
                    transitions = self.get_transition_probs(s, a)
                    for (prob, next_s, reward) in transitions:
                        q_val += prob * (reward + self.gamma * self.V[next_s])
                    q_values.append(q_val)

                best_val = max(q_values)
                delta = max(delta, abs(best_val - self.V[s]))
                new_V[s] = best_val

            self.V = new_V
            iteration += 1
            if iteration % 5 == 0:
                print(f"{iteration:<10} | {delta:<15.5f}")

            if delta < epsilon:
                break

        print(f"\nConverged in {iteration} iterations.")

    def get_optimal_policy(self):
        print("\n--- OPTIMAL MDP POLICY (Selected Examples) ---")
        print(f"{'State Description':<40} | {'Best Action'}")
        print("-" * 65)


        examples = [
            (0, 9, 0),
            (2, 9, 0),
            (4, 9, 1),
            (5, 5, 1),
            (8, 9, 1),
            (8, 2, 2),
            (9, 1, 2)
        ]

        for h, r, s in examples:
            sid = get_state_id(h, r, s)
            q_values = []
            for a in range(self.num_actions):
                q_val = 0
                transitions = self.get_transition_probs(sid, a)
                for (prob, next_s, reward) in transitions:
                    q_val += prob * (reward + self.gamma * self.V[next_s])
                q_values.append(q_val)

            best_action = ACTIONS[np.argmax(q_values)]
            desc = f"H:{h}/9, Res:{r}/9, Stat:{['NoDmg','Threat','Impact','Evac'][s]}"
            print(f"{desc:<40} | {best_action}")

if __name__ == "__main__":
    model = DisasterMDPModel()
    model.value_iteration()
    model.get_optimal_policy()

Solving MDP with 400 states...
Iteration  | Max Change     
------------------------------
5          | 4.97586        
10         | 2.48589        
15         | 1.43433        
20         | 0.81558        
25         | 0.46269        
30         | 0.26242        
35         | 0.14883        
40         | 0.08441        
45         | 0.04787        
50         | 0.02715        
55         | 0.01549        
60         | 0.00893        
65         | 0.00515        
70         | 0.00297        
75         | 0.00171        
80         | 0.00098        

Converged in 80 iterations.

--- OPTIMAL MDP POLICY (Selected Examples) ---
State Description                        | Best Action
-----------------------------------------------------------------
H:0/9, Res:9/9, Stat:NoDmg               | DelayAndObserve
H:2/9, Res:9/9, Stat:NoDmg               | DelayAndObserve
H:4/9, Res:9/9, Stat:Threat              | PreemptiveRelocation
H:5/9, Res:5/9, Stat:Threat              | PreemptiveRelocation
H

In [ ]:
import numpy as np
import random

ACTIONS = ["ImmediateEvacuation", "DelayAndObserve", "SendDroneFirst", "PreemptiveRelocation"]
A_EVAC = 0
A_DELAY = 1
A_DRONE = 2
A_RELOC = 3


MAX_HAZARD = 9
MAX_RESOURCES = 9
STATUS_LEVELS = 4

H_HIGH_THRESHOLD = 7
H_MED_THRESHOLD = 3


STATES = []
for h in range(MAX_HAZARD + 1):
    for r in range(MAX_RESOURCES + 1):
        for s in range(STATUS_LEVELS):
            STATES.append((h, r, s))

def get_state_id(h, r, s):
    h = max(0, min(MAX_HAZARD, h))
    r = max(0, min(MAX_RESOURCES, r))
    s = max(0, min(STATUS_LEVELS - 1, s))
    return (h * (MAX_RESOURCES + 1) * STATUS_LEVELS) + (r * STATUS_LEVELS) + s

class DisasterQLearningAgent:
    def __init__(self):
        self.num_states = len(STATES)
        self.num_actions = len(ACTIONS)
        self.q_table = np.zeros((self.num_states, self.num_actions))
        self.learning_rate = 0.1
        self.discount_factor = 0.9

    def get_reward(self, state_tuple, action, next_state_tuple):
        h, r, s = state_tuple
        nh, nr, ns = next_state_tuple

        reward = 0

        if ns == 3 and s != 3:
            if h >= H_HIGH_THRESHOLD: reward += 100
            elif h >= H_MED_THRESHOLD: reward += 80
            else: reward += 10

            if action == A_RELOC and h >= H_MED_THRESHOLD and h < H_HIGH_THRESHOLD:
                reward += 20


        if nr < r:
            reward -= (r - nr) * 5


        if action == A_DELAY:
            if h >= H_HIGH_THRESHOLD: reward -= 50
            if h < H_MED_THRESHOLD: reward += 5
            if nh > h: reward -= 20

        if action == A_DRONE:
            reward -= 2
            if h >= H_MED_THRESHOLD and h < H_HIGH_THRESHOLD: reward += 5

        if nh >= H_HIGH_THRESHOLD and ns == 2: reward -= 50
        if nr == 0 and nh > 0: reward -= 100

        return reward

    def step_simulated(self, state_id, action):

        h, r, s = STATES[state_id]


        if s == 3: return state_id, 0, True
        if r == 0 and h >= H_HIGH_THRESHOLD: return state_id, -100, True

        nh, nr, ns = h, r, s


        if action == A_EVAC:
            nr = max(0, r - 3)
            ns = 3

        elif action == A_DELAY:
            roll = random.random()
            if h < H_MED_THRESHOLD:

                if roll < 0.1:
                    nh = min(MAX_HAZARD, h + 1)
                    if nh >= H_MED_THRESHOLD: ns = 1
                elif roll < 0.3:
                    nh = 0

            else:

                if roll < 0.4:
                    nh = min(MAX_HAZARD, h + 1)
                    if nh >= H_HIGH_THRESHOLD and s == 1: ns = 2
                elif roll > 0.8:
                    nh = max(0, h - 1)

        elif action == A_DRONE:
            nr = max(0, r - 1)

            if random.random() < 0.3:
                nh = max(0, h - 1)

        elif action == A_RELOC:
            if s == 1 or (h >= H_MED_THRESHOLD and h < H_HIGH_THRESHOLD):
                nr = max(0, r - 2)
                ns = 3
            else:
                nr = max(0, r - 2)

        next_state_id = get_state_id(nh, nr, ns)
        next_tuple = STATES[next_state_id]
        reward = self.get_reward((h,r,s), action, next_tuple)

        done = (ns == 3) or (nr == 0 and nh >= H_HIGH_THRESHOLD)
        return next_state_id, reward, done

    def generate_random_data(self, num_samples=200):

        print(f"Generating {num_samples} random data points (transitions)...")
        dataset = []

        for _ in range(num_samples):

            valid_start = False
            while not valid_start:
                sid = random.randint(0, self.num_states - 1)
                if STATES[sid][2] != 3:
                    valid_start = True


            action = random.choice([0, 1, 2, 3])

            next_sid, reward, done = self.step_simulated(sid, action)

            dataset.append((sid, action, reward, next_sid))

        return dataset

    def train_on_data(self, dataset, epochs=50):

        print(f"Training on dataset for {epochs} epochs...")

        for epoch in range(epochs):
            total_change = 0
            for (state, action, reward, next_state) in dataset:
                old_val = self.q_table[state, action]
                next_max = np.max(self.q_table[next_state])


                new_val = (1 - self.learning_rate) * old_val + \
                          self.learning_rate * (reward + self.discount_factor * next_max)

                self.q_table[state, action] = new_val
                total_change += abs(new_val - old_val)


            print(f"Epoch {epoch:<3} | Avg Q-Change: {total_change/len(dataset):.4f}")

    def get_policy_summary(self):
        print("\n--- COMPLETE OPTIMAL POLICY (All 400 States) ---")
        print(f"{'State Description':<45} | {'Best Action'}")
        print("-" * 70)


        for h in range(MAX_HAZARD + 1):
            for r in range(MAX_RESOURCES + 1):
                for s in range(STATUS_LEVELS):
                    sid = get_state_id(h, r, s)

                    best_action_idx = np.argmax(self.q_table[sid])
                    best_action = ACTIONS[best_action_idx]


                    status_str = ['NoDmg', 'Threat', 'Impact', 'Evac'][s]
                    desc = f"H:{h:<2} R:{r:<2} S:{status_str}"

                    if s == 3:
                        print(f"{desc:<45} | (Already Safe)")
                    elif r == 0 and h >= H_HIGH_THRESHOLD:
                        print(f"{desc:<45} | (Failed/Crisis)")
                    else:
                        print(f"{desc:<45} | {best_action}")

if __name__ == "__main__":
    agent = DisasterQLearningAgent()


    data = agent.generate_random_data(num_samples=200)

    agent.train_on_data(data, epochs=50)


    agent.get_policy_summary()

Generating 200 random data points (transitions)...
Training on dataset for 50 epochs...
Epoch 0   | Avg Q-Change: 4.0819
Epoch 1   | Avg Q-Change: 3.6096
Epoch 2   | Avg Q-Change: 3.2087
Epoch 3   | Avg Q-Change: 2.8651
Epoch 4   | Avg Q-Change: 2.5702
Epoch 5   | Avg Q-Change: 2.3132
Epoch 6   | Avg Q-Change: 2.0843
Epoch 7   | Avg Q-Change: 1.8802
Epoch 8   | Avg Q-Change: 1.6981
Epoch 9   | Avg Q-Change: 1.5355
Epoch 10  | Avg Q-Change: 1.3906
Epoch 11  | Avg Q-Change: 1.2608
Epoch 12  | Avg Q-Change: 1.1449
Epoch 13  | Avg Q-Change: 1.0411
Epoch 14  | Avg Q-Change: 0.9481
Epoch 15  | Avg Q-Change: 0.8649
Epoch 16  | Avg Q-Change: 0.7904
Epoch 17  | Avg Q-Change: 0.7234
Epoch 18  | Avg Q-Change: 0.6632
Epoch 19  | Avg Q-Change: 0.6088
Epoch 20  | Avg Q-Change: 0.5599
Epoch 21  | Avg Q-Change: 0.5157
Epoch 22  | Avg Q-Change: 0.4759
Epoch 23  | Avg Q-Change: 0.4400
Epoch 24  | Avg Q-Change: 0.4075
Epoch 25  | Avg Q-Change: 0.3782
Epoch 26  | Avg Q-Change: 0.3517
Epoch 27  | Avg Q-Cha